# ch01.11 CNN실전 - 랜드마크 이미지

* 날짜:
* 이름:

## 학습내용
    - 랜드마크 데이터를 이용한 적절한 모델을 구축 및 학습한다.

### 소개

인공지능 기반의 컴퓨터 비전 기술 및 서비스 개발에 활용하기 위해 국내 특성(지리 공간적, 기능적)이 반영된 국내 도심 민간건물, 공공기관, 관광명소, 편의시설 등 국내 도시별 주요 랜드마크 이미지 데이터 구축

### 구축목적
AI 기술 및 응용서비스 개발에 활용가치가 높은 인공지능 학습용 데이터 구축 및 개방, AI응용 서비스 개발
국내 특성(지리 공간적, 기능적)이 반영된 국내 도심 민간건물, 공공기관, 관광명소, 편의시설 등 국내 도시별 주요 랜드마크 이미지 데이터 구축

### AI허브 링크
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=56

## 실습
---

In [1]:
import datetime, os, time, pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses, models, layers, optimizers, callbacks, utils, metrics
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
data_dir = pathlib.Path("D:Dataset/랜드마크 이미지/Training/")
image_count = len(list(data_dir.glob('*/*.jpg'))) # 총 이미지 개수
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [4]:
val_size = int(image_count * 0.15)
train_ = list_ds.skip(val_size) # 20% 를 넘김
valid_ = list_ds.take(val_size) # 20% 를 가짐 
NUM_TRAIN = tf.data.experimental.cardinality(train_).numpy()
NUM_VALID = tf.data.experimental.cardinality(valid_).numpy()
print(tf.data.experimental.cardinality(train_).numpy())
print(tf.data.experimental.cardinality(valid_).numpy())

10537
1859


In [10]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))

In [5]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    one_hot = parts[-2] == class_names
    return tf.argmax(one_hot)

In [6]:
def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    return tf.image.resize(img, [img_height, img_width])

In [7]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [8]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [9]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(img_height, img_width, 3))
    y = base_model(x)

    # Rebuild
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    top_dropout_rate = 0.2
    y = layers.Dropout(top_dropout_rate, name="top_dropout")(y)
    y = layers.Dense(84, activation="softmax", name="Pred")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=0.0001)
    loss = losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=["acc"])
    return model

## 모델 학습

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
NAME = ["B0", "B1", "B2", "B3", "B4","B5","B6","B7"]
MODEL = [EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7]

for name,M in zip(NAME, MODEL):
    SIZE = 180
    img_height = SIZE
    img_width = SIZE
    image_size=(img_height, img_width)
    train_ds = train_.map(process_path, num_parallel_calls=AUTOTUNE)
    valid_ds = valid_.map(process_path, num_parallel_calls=AUTOTUNE)
    train_ds = configure_for_performance(train_ds)
    valid_ds = configure_for_performance(valid_ds)
    base_model = M(input_shape=(img_height,img_width,3), include_top=False, weights='imagenet')
    model = build_eff_model()
    ckp = callbacks.ModelCheckpoint(f"model/model_{name}.h5", save_best_only=True,  monitor="val_acc")
    early_stopping = callbacks.EarlyStopping(monitor="val_acc", patience=5)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_acc", patience=3, factor=0.1)
    model.fit(train_ds,epochs=10,validation_data=valid_ds,callbacks=[ckp, early_stopping, reduce_lr])

## 추론

In [12]:
import pandas as pd

In [15]:
def configure_for_test(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [16]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
result = pd.DataFrame([])
result_conf = pd.DataFrame([])

data_dir = pathlib.Path("D:Dataset/랜드마크 이미지/Validation/")
image_count = len(list(data_dir.glob('*/*.jpg'))) # 총 이미지 개수
test_ = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)

model_dir = "C:/Users/user/OneDrive/document/GitHub/Prac_Project/dacon/랜드마크/model/"

for name in ["B0", "B1", "B2", "B3", "B4","B5","B6","B7"]:
    SIZE = 180
    img_height = SIZE
    img_width = SIZE
    image_size=(img_height, img_width)
    test_ds = test_.map(process_path, num_parallel_calls=AUTOTUNE)
    test_ds = configure_for_test(test_ds)
    model = tf.keras.models.load_model(model_dir + f"model_{name}.h5")
    a = model.predict(test_ds)
    arg = np.argmax(a,axis=1)
    conf = []
    for i, j in zip(a, arg):
        conf.append(i[j])
    result[f"size_{name}"] = arg
    result_conf[f"size_{name}_conf"] = conf

In [17]:
result["answer"] = result.mode(axis=1)[0]
result["answer"] = result["answer"].astype("int")
result_conf["answer"] = result_conf.mean(axis=1)

test_ds = test_.map(process_path, num_parallel_calls=AUTOTUNE)
temp = []
for i, j in test_ds:
    temp.append(j.numpy())
result["Real"] = temp

## 평가

In [20]:
def GAP_vector(pred, conf, true, return_x=False):
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending=False, inplace=True, na_position='last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

In [21]:
gap , x = GAP_vector(result["answer"], result_conf["answer"], result["Real"], return_x=True)

In [22]:
gap

0.997992893767759

In [23]:
x

,pred,conf,true,correct,prec_k,term
983,54,0.999994,54,1,1.000000,1.000000
1025,55,0.999993,55,1,1.000000,1.000000
471,25,0.999990,25,1,1.000000,1.000000
984,54,0.999990,54,1,1.000000,1.000000
197,11,0.999989,11,1,1.000000,1.000000
...,...,...,...,...,...,...
109,7,0.591827,7,1,0.998000,0.998000
1251,67,0.548511,67,1,0.998001,0.998001
1253,67,0.546929,67,1,0.998003,0.998003
111,7,0.538739,7,1,0.998004,0.998004
